In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm




# SP500 version

In [ ]:
quote ='https://finance.yahoo.com/quote/'
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500 =sp500[0]
sp500 = sp500[['Security','Symbol']]
for i in sp500.index:
    if ".B" in sp500.loc[i,'Symbol']:
        sp500 = sp500.drop(i)

In [ ]:
#newer version 
l = []
counter = 0
for i in tqdm(sp500['Symbol']):
    counter +=1
    qurl = quote+i
    price = pd.read_html(qurl)
    try:
        chain = pd.read_html(quote+i+"/options")[0]
    except:
        print('none')
        l.append(0)
        continue
    
    
    bid_stock = float(price[0].loc[2,1][:5].replace(',',''))
    ask_stock = float(price[0].loc[3,1][:5].replace(',',''))
    
    price = (float(bid_stock)+float(ask_stock))/2
    
    strike_prices = list(chain['Strike'])
    
    chosen_strike= min(strike_prices, key=lambda x:abs(x-price))
    
    try:
        bid_option = float(chain[chain['Strike'] == chosen_strike]['Bid'])
        ask_option = float(chain[chain['Strike'] == chosen_strike]['Ask'])
    except:
        l.append(0)
        print('none')
        continue

    option_price = (bid_option+ask_option)/2
    
    try:
        perc = (price-(chosen_strike+option_price))/price
    except:
        perc = 0 
    
    l.append(perc)
    

    
l = [x * 100 for x in l]
sp500['percent'] = l

    

In [ ]:
#old version finds movers!!!
li = []
counter = 0
for i in sp500['Symbol']:
    counter +=1
    qurl = quote+i
    price = pd.read_html(qurl)[0].loc[1,1]
    try:
        chain = pd.read_html(quote+i+"/options")
        calls = chain[0]
    except:
        print('none')
        li.append(0)
        continue
    strike = round(float(price),0)
    e = calls[calls['Strike'] == strike]['Last Price']
    c = 0
    for i in e:
        c= i
    perc = c/float(price)
    
    li.append(perc)
    
    
li = [x * 100 for x in li]
sp500['percent'] = li


    

In [8]:
#filter above 2 percent profitability 
filt = sp500[sp500['percent']>2]

# NASDAQ version

In [13]:
#fetching the data 
nasdaq = pd.read_html('https://en.wikipedia.org/wiki/NASDAQ-100')
nasdaq = nasdaq[3]
nasdaq

,Company,Ticker
0,Activision Blizzard,ATVI
1,Adobe Inc.,ADBE
2,Advanced Micro Devices,AMD
3,Alexion Pharmaceuticals,ALXN
4,Align Technology,ALGN
...,...,...
98,"Walgreen Boots Alliance, Inc.",WBA
99,"Workday, Inc.",WDAY
100,"Xcel Energy, Inc.",XEL
101,"Xilinx, Inc.",XLNX


In [14]:
for i in nasdaq.index:
    if ".B" in nasdaq.loc[i,'Ticker']:
        nasdaq = nasdaq.drop(i)

In [ ]:
l = []
counter = 0
for i in nasdaq['Ticker']:
    counter +=1
    qurl = quote+i
    price = pd.read_html(qurl)
    try:
        chain = pd.read_html(quote+i+"/options")[0]
    except:
        print('none')
        l.append(0)
        continue
    
    
    bid_stock = float(price[0].loc[2,1][:5].replace(',',''))
    ask_stock = float(price[0].loc[3,1][:5].replace(',',''))
    
    price = (float(bid_stock)+float(ask_stock))/2
    
    strike_prices = list(chain['Strike'])
    
    chosen_strike= min(strike_prices, key=lambda x:abs(x-price))
    
    
    bid_option = float(chain[chain['Strike'] == chosen_strike]['Bid'])
    ask_option = float(chain[chain['Strike'] == chosen_strike]['Ask'])
    
    option_price = (bid_option+ask_option)/2
    
    
   
    perc = option_price/price
    
    l.append(perc)
    
    print(counter)
    
l = [x * 100 for x in l]
nasdaq['percent'] = l

    

In [34]:
nasdaq[nasdaq['percent']>3]

,Company,Ticker,percent
33,CoStar Group,CSGP,3.478261
40,Expedia Group,EXPE,4.126984
71,NVIDIA Corporation,NVDA,3.819188


# For all stocks in the NYSE 

In [ ]:
#Version Gets all stocks from the NYSE

import string 

alphabet = string.ascii_uppercase

frame_list = []
for i in alphabet:
    stock_list = 'https://en.wikipedia.org/wiki/Companies_listed_on_the_New_York_Stock_Exchange_('+i+')'
    check = pd.read_html(stock_list)
    frame_list.append(check[1])
    

result = pd.concat(frame_list)


In [ ]:
result = result[result['Country of origin']=='US']
result.apply(lambda x: '.' in  x['Symbol'] ,axis=1)
bool = result.apply(lambda x: '.' in x['Symbol'], axis = 1)
result['bool']  = bool
test = result[result['bool'] == False]

In [ ]:
test = test[['Stock name','Symbol']]

In [ ]:
p = []
counter = 0


for i in test['Symbol']:
    counter +=1
    qurl = quote+i
    try:
        price = pd.read_html(qurl)[0].loc[1,1]
    except:
        print("none")
        p.append(0)
    
    try:
        chain = pd.read_html(quote+i+"/options")
        calls = chain[0]
    except:
        p.append(0)
        print('none')
        continue        
        
    try:
        strike = round(float(price),1)
        e = calls[calls['Strike'] == strike]['Last Price']
    except:
        p.append(0)
        print('none2')
        continue
        
        
        
    c = 0
    for i in e:
        c= i
        
        
    try:
        perc = c/float(price)
    except:
        perc = 0 
        continue
        
        
        
    
    p.append(perc)
    
    print(counter)
    
p = [x * 100 for x in p]
test['percent'] = p


In [12]:
part2 = p[:2239]

In [13]:
part['perc']= part2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
out = part[part['perc']>5]

In [15]:
out =out[out['perc']<10]

In [16]:
out

,Stock name,Symbol,perc
14,Callon Petroleum Co.,CPE,6.600660
36,Capital Senior Living Corporation,CSU,5.723906
101,Cherry Hill Mortgage Investment Corporation,CHMI,9.115770
24,The Denali Fund Inc.,DNY,6.796117
38,DiamondRock Hospitality Company,DRH,8.196721
102,"Envision Healthcare Holdings, Inc.",EVHC,9.920635
78,FMC Corporation,FMC,6.191038
34,"Oneok Partners, L.P.",OKS,7.792208
5,"QTS Realty Trust, Inc.",QTS,6.930693


In [17]:
print(out.to_string())

                                      Stock name Symbol      perc
14                          Callon Petroleum Co.    CPE  6.600660
36             Capital Senior Living Corporation    CSU  5.723906
101  Cherry Hill Mortgage Investment Corporation   CHMI  9.115770
24                          The Denali Fund Inc.    DNY  6.796117
38               DiamondRock Hospitality Company    DRH  8.196721
102           Envision Healthcare Holdings, Inc.   EVHC  9.920635
78                               FMC Corporation    FMC  6.191038
34                          Oneok Partners, L.P.    OKS  7.792208
5                         QTS Realty Trust, Inc.    QTS  6.930693


In [63]:
out

,Stock name,Symbol,perc
22,Adeptus Health Inc.,ADPT,5.270092
123,"American Axle & Manufacturing Holdings, Inc.",AXL,9.159483
175,Antero Resources Corporation,AR,6.470588
176,Antero Midstream Partners LP,AM,6.862745
287,Axalta Coating Systems Ltd.,AXTA,5.801688
62,Belden Inc.,BDC,5.430410
14,Callon Petroleum Co.,CPE,8.196721
36,Capital Senior Living Corporation,CSU,7.092199
57,"CBL & Associates Properties, Inc.",CBL,5.700604
82,Central Pacific Financial Corp.,CPF,6.960557
